<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/C3669C-2025-01/blob/main/L06/L06.ipynb)

# Setup and Installation

You can run this Jupyter notebook either on your local machine or run it at Google Colab.

* For local machine, it is recommended to install Anaconda and create a new development environment called `c3669c`.
* Pip/Conda install the libraries stated below when necessary.
---

# <font color='red'>ATTENTION</font>

## Google Colab
- If you are running this code in Google Colab, **DO NOT** store the API Key in a text file and load the key later from Google Drive. This is insecure and will expose the key.
- **DO NOT** hard code the API Key directly in the Python code, even though it might seem convenient for quick development.
- You need to enter the API key at python code `getpass.getpass()` when ask.

## Local Environment/Laptop
- If you are running this code locally in your laptop, you can create a env.txt and store the API key there.
- Make sure env.txt is in the same directory of this Jupyter notebook.
- You need to install `python-dotenv` and run the Python code to load in the API key.

---
```
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv('env.txt')
openai_api_key = os.getenv('OPENAI_API_KEY')
```
---

## GitHub/GitLab
- **DO NOT** `commit` or `push` API Key to services like GitHub or GitLab.

# Lesson 06

Hugging Face is a cutting-edge technology company that has become a cornerstone in the field of artificial intelligence (AI) and natural language processing (NLP). Founded in 2016, Hugging Face is best known for its open-source contributions, particularly the Transformers library, which has revolutionized how researchers and developers work with state-of-the-art machine learning models. The company’s mission is to democratize AI by making advanced tools and models accessible to everyone, from students and hobbyists to industry professionals.  

At its core, Hugging Face provides a platform for sharing, training, and deploying machine learning models, with a strong focus on NLP tasks such as text classification, translation, summarization, and question answering. The platform hosts thousands of pre-trained models, datasets, and tools that enable users to build AI applications without starting from scratch. This has made Hugging Face a go-to resource for anyone interested in exploring or advancing AI technologies.

Hugging Face is not just a tool but a vibrant ecosystem that empowers individuals to innovate and collaborate in the rapidly evolving world of AI. As you embark on your journey in AI and machine learning, Hugging Face is a platform that can help you turn theoretical knowledge into practical, impactful solutions

---

# Requirements

- Go to [Hugging Face](https://huggingface.co/) to sign up for an account.
- Create an access token in `Hugging Face`.
- You can store the access token in Colab by adding a secret.

<img src="Colab Secrets.png" width="auto" height="auto">

In [ ]:
%%capture --no-stderr

%pip install --quiet -U transformers
%pip install --quiet -U gradio

# uncomment to either install
# pytorch or tensorflow

# if you are running this code in Colab
# it is not necessary to install either
#%pip install --quiet -U torch
#%pip install --quiet -U tensorflow

# Using Pipeline

## Sentiment Classifier:

In [1]:
# model card: https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
from transformers import pipeline

# load a pretrained sentiment analysis_model
sentiment_classifier = pipeline(
    "sentiment-analysis",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)

In [2]:
text = "I love using Hugging Face!"

result = sentiment_classifier(text)

print(result)

[{'label': 'POSITIVE', 'score': 0.9997085928916931}]


In [3]:
text = "You have to do better in NLP."

result = sentiment_classifier(text)

print(result)

[{'label': 'NEGATIVE', 'score': 0.9995294809341431}]


## Translation:

In [5]:
# model card: https://huggingface.co/facebook/nllb-200-distilled-600M
import torch

translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    torch_dtype=torch.bfloat16 
)

text = """
The Merlion is a famous symbol of Singapore. It has the head of a lion and the body of a fish. \
The lion represents Singapore's original name, "Singapura," meaning "Lion City," \
while the fish body shows its history as a fishing village. The Merlion statue is a popular tourist spot!
"""

# check https://huggingface.co/facebook/nllb-200-distilled-600M/blob/main/README.md
# for language details
text_translated = translator(
    text,
    src_lang="eng_Latn",
    tgt_lang="zho_Hans"
)

print(f"\nOriginal: {text}")
print(f"Translation: {text_translated[0]['translation_text']}")


Original: 
The Merlion is a famous symbol of Singapore. It has the head of a lion and the body of a fish. The lion represents Singapore's original name, "Singapura," meaning "Lion City," while the fish body shows its history as a fishing village. The Merlion statue is a popular tourist spot!

Translation: 狮子代表新加坡原名"新加坡"意思是"狮子城",而鱼体则显示其作为渔村的历史.


# Using the Model Directly

Pytorch Example

In [6]:
# model card: https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [7]:
# Encode text
text = "I love programming!"
tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokens)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

label_ids = torch.argmax(probabilities, dim=1)

labels = ['Negative', 'Positive']
label = labels[label_ids]

print(f"The sentiment is: {label}")

The sentiment is: Positive


In [8]:
# Encode text
text = "I hate programming!"
tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokens)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

label_ids = torch.argmax(probabilities, dim=1)

labels = ['Negative', 'Positive']
label = labels[label_ids]

print(f"The sentiment is: {label}")

The sentiment is: Negative


# Fill Mask

In [9]:
# model card: https://huggingface.co/distilbert/distilbert-base-uncased
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')

unmasker("Hello I'm a [MASK] model.")

[{'score': 0.052928801625967026,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i ' m a role model."},
 {'score': 0.03968583047389984,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i ' m a fashion model."},
 {'score': 0.03474362939596176,
  'token': 2449,
  'token_str': 'business',
  'sequence': "hello i ' m a business model."},
 {'score': 0.03462280333042145,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i ' m a model model."},
 {'score': 0.01814517006278038,
  'token': 11643,
  'token_str': 'modeling',
  'sequence': "hello i ' m a modeling model."}]

# Text Summarization

In [10]:
# model card: https://huggingface.co/facebook/bart-large-cnn
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Source: https://en.wikipedia.org/wiki/Quantum_computing
Article = """A quantum computer is a computer that exploits quantum mechanical phenomena. 
On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specialized hardware. 
Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any modern "classical" computer. 
Theoretically a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing physical simulations; 
however, the current state of the art is largely experimental and impractical, with several obstacles to useful applications.

The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. 
However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, 
a state that is in an abstract sense "between" the two basis states. When measuring a qubit, the result is a probabilistic output of a classical bit. 
If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. 
The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.

Quantum computers are not yet practical for real work. Physically engineering high-quality qubits has proven challenging. 
If a physical qubit is not sufficiently isolated from its environment, it suffers from quantum decoherence, introducing noise into calculations. 
National governments have invested heavily in experimental research that aims to develop scalable qubits with longer coherence times and lower error rates. 
Example implementations include superconductors (which isolate an electrical current by eliminating electrical resistance) and ion traps (which confine a single atomic particle using electromagnetic fields).

In principle, a classical computer can solve the same computational problems as a quantum computer, given enough time. 
Quantum advantage comes in the form of time complexity rather than computability, 
and quantum complexity theory shows that some quantum algorithms are exponentially more efficient than the best-known classical algorithms. 
A large-scale quantum computer could in theory solve computational problems unsolvable by a classical computer in any reasonable amount of time. 
This concept of extra ability has been called "quantum supremacy". 
While such claims have drawn significant attention to the discipline, near-term practical use cases remain limited.

"""

In [11]:
Output = summarizer(Article, max_length=200, min_length=50, do_sample=False)

print(f"Summary: {Output[0]['summary_text']}")

Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. A large-scale quantum computer could in theory solve computational problems unsolvable by a classical computer in any reasonable amount of time. The current state of the art is largely experimental and impractical, with several obstacles to useful applications.


# Question and Answering

Using `Roberta` model as a Question and Answering chatbot. We will provide context and query the model.

In [13]:
# model card: https://huggingface.co/deepset/roberta-base-squad2
model_name = "deepset/roberta-base-squad2"

nlp = pipeline(
    'question-answering', 
    model=model_name, 
    tokenizer=model_name
)

In [14]:
QA_Input = {
    'question' : 'What is the current (2025) Enhanced Retirement Sum (ERS) in dollar?',
    'context' : """The ERS is increased yearly. For members 55 and above who wish to have higher monthly payouts in retirement, they can top up their Retirement Account up to the current ERS. To find out how much you can top up, log in to your Retirement dashboard.

From 2025, the ERS was raised from 3 times the Basic Retirement Sum (BRS) to 4 times to enable members to voluntarily commit more savings for even higher payouts. 

This allows members to get higher CPF LIFE monthly payouts of up to $3,300 for life from age 65 through CPF top-ups. 
    
Here are the ERS sums from years 2025 to 2027 to enable you to plan your top-ups:
    2025: $426,000
    2026: $440,800
    2027: $456,400"""
}

response = nlp(question=QA_Input['question'], context=QA_Input['context'])

print(f"Question: {QA_Input['question']}")
print(f"Answer: {response['answer']}")

C:\Users\koay_seng_tian\AppData\Local\anaconda3\envs\c3669c\lib\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Question: What is the current (2025) Enhanced Retirement Sum (ERS) in dollar?
Answer: $426,000


---

# Deployment (Local)

Deploying large language models (LLMs) from Hugging Face has become incredibly accessible thanks to tools like Streamlit and Gradio. These frameworks allow you to quickly create interactive web applications for showcasing and interacting with AI models, making them ideal for students, researchers, and developers.

Below is a screen shot of the deployment done on local machine.

<img src="Gradio Screenshot 01.png" width="auto" height="auto">

In [ ]:
import gradio as gr

from transformers import pipeline

model_name = "deepset/roberta-base-squad2"

In [ ]:
nlp = pipeline(
   'question-answering', 
    model=model_name, 
    tokenizer=model_name
)

In [ ]:
def generate_answer(question, context):
    QA_Input = {
        'question': question,
        'context': context      
    }

    response = nlp(question=question, context=context)
    result = response['answer']

    return result

In [ ]:
iface = gr.Interface(
    fn=generate_answer, 
    
    inputs=[
        gr.Textbox(label="Question:", placeholder="Enter your question here..."),
        gr.Textbox(label="Context:", placeholder="Enter the context here...")
    ],

    outputs=gr.Textbox(label="Answer:"),
    
    title="Question and Answering",
    description="Enter a question and a context and the system will provide an answer."
)

iface.launch(server_name="0.0.0.0", server_port=7860)

---

# Deployment (Hugging Face Space)

Refer to lesson PowerPoint for instructions on how to setup the Gradio app to run in Hugging Face Space.

<img src="Gradio Screenshot HFS 99.png" width="auto" height="auto">

You will need the files:
- [app.py](https://github.com/koayst-rplesson/C3669C-2025-01/blob/main/L06/app.py)
- [requirements.txt](https://github.com/koayst-rplesson/C3669C-2025-01/blob/main/L06/requirements.txt)
- [Dockerfile](https://github.com/koayst-rplesson/C3669C-2025-01/blob/main/L06/Dockerfile)

The Gradio app is hosted in Hugging Face Space using the `Docker container` method.